In [1]:
import numpy as np

In [2]:
pi_x = np.load("pi_train_data_ann.npy")
pi_y = np.load("pi_train_label_ann.npy")
z_x = np.load("z_train_data_ann.npy")
z_y = np.load("z_train_label_ann.npy")

In [3]:
print (np.shape(pi_x),np.shape(pi_y),np.shape(z_x),np.shape(z_y))

(1717, 1, 10, 1) (1717, 9) (1717, 1, 10, 1) (1717, 1)


In [4]:
print pi_x[600],pi_y[600],z_x[600],z_y[600]

[[[ 0.]
  [ 0.]
  [ 0.]
  [ 2.]
  [ 1.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 2.]]] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.] [[[ 0.]
  [ 0.]
  [ 0.]
  [ 2.]
  [ 1.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 2.]]] [-1.]


In [2]:
def custom_loss_function(y_true, y_pred):

	p = y_pred
	pi = y_true

	zero = tf.zeros(shape = tf.shape(pi), dtype=tf.float32)
	where = tf.equal(pi, zero)

	negatives = tf.fill(tf.shape(pi), -100.0) 
	p = tf.where(where, negatives, p)

	loss = tf.nn.softmax_cross_entropy_with_logits(labels = pi, logits = p)

	return loss


In [3]:
import os 
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
import time
import math
import numpy as np
import keras
import tensorflow as tf
import os.path
from keras.layers import Dense, Activation, Conv2D, BatchNormalization, LeakyReLU, Flatten, merge, Input, ELU, concatenate
from keras.models import load_model, Model
from keras.optimizers import SGD
from keras import regularizers

Using TensorFlow backend.
/home/talha/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/talha/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/talha/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/talha/anaconda3/lib/python3.7/site-packages/tensorflo

In [4]:
from keras.utils import plot_model

In [5]:
def conv_layer(_in):


    conv1 = Conv2D(256, (3, 3), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    bn1 = BatchNormalization()(conv1)
    
    lr1 = ELU()(bn1)

    return lr1


def residual_layer(_in):

    conv1 = Conv2D(256, (3, 3), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    bn1 = BatchNormalization()(conv1)
    
    lr1 = ELU()(bn1)
    
    conv2 = Conv2D(256, (3, 3), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(lr1)

    bn2 = BatchNormalization()(conv2)
    
    m1 = concatenate([_in, bn2])
    
    lr2 = ELU()(m1)
    
    
    return lr2




def value_head(_in):


    conv1 = Conv2D(1, (1, 1), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    bn1 = BatchNormalization()(conv1)

    lr1 = ELU()(bn1)

    f1 = Flatten()(lr1)

    d1 = Dense(20, activation='linear', kernel_regularizer=regularizers.l2(0.0001))(f1)

    lr2 = ELU()(d1)

    d2 = Dense(1, use_bias=False, activation='tanh', kernel_regularizer=regularizers.l2(0.0001))(lr2)

    return d2

def policy_head(_in):


    conv1 = Conv2D(2, (1, 1), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    bn1 = BatchNormalization()(conv1)

    lr1 = ELU()(bn1)

    f1 = Flatten()(lr1)

    d1 = Dense(9, activation='linear', kernel_regularizer=regularizers.l2(0.0001))(f1)

    return d1



In [6]:
def model_z():
    inputs = Input(shape=(1, 10, 1))
    
    conv1 = conv_layer(inputs)
    
    r1 = residual_layer(conv1)
    r2 = residual_layer(r1)
    r3 = residual_layer(r2)
    r4 = residual_layer(r3)
    r5 = residual_layer(r4)
    r6 = residual_layer(r5)
    r7 = residual_layer(r6)
    r8 = residual_layer(r7)
    r9 = residual_layer(r8)
    r10 = residual_layer(r9)

    r11 = residual_layer(r10)
    r12 = residual_layer(r11)
    r13 = residual_layer(r12)
    r14 = residual_layer(r13)
    r15 = residual_layer(r14)
    r16 = residual_layer(r15)
    r17 = residual_layer(r16)
    r18 = residual_layer(r17)
    r19 = residual_layer(r18)
    r20 = residual_layer(r19)

    r21 = residual_layer(r20)
    r22 = residual_layer(r21)
    r23 = residual_layer(r22)
    r24 = residual_layer(r23)
    r25 = residual_layer(r24)
    r26 = residual_layer(r25)
    r27 = residual_layer(r26)
    r28 = residual_layer(r27)
    r29 = residual_layer(r28)
    r30 = residual_layer(r29)


    r31 = residual_layer(r30)
    r32 = residual_layer(r31)
    r33 = residual_layer(r32)
    r34 = residual_layer(r33)
    r35 = residual_layer(r34)
    r36 = residual_layer(r35)
    r37 = residual_layer(r36)
    r38 = residual_layer(r37)
    r39 = residual_layer(r38)
    r40 = residual_layer(r39)

    output = value_head(r40)



    model = Model(inputs=[inputs], outputs=[output])

    model.compile(loss='mean_squared_error', optimizer=SGD(lr=0.1, momentum = 0.9))
    
    return model


In [7]:
def model_pi():
    inputs = Input(shape=(1, 10, 1))
    
    conv1 = conv_layer(inputs)
    
    r1 = residual_layer(conv1)
    r2 = residual_layer(r1)
    r3 = residual_layer(r2)
    r4 = residual_layer(r3)
    r5 = residual_layer(r4)
    r6 = residual_layer(r5)
    r7 = residual_layer(r6)
    r8 = residual_layer(r7)
    r9 = residual_layer(r8)
    r10 = residual_layer(r9)

    r11 = residual_layer(r10)
    r12 = residual_layer(r11)
    r13 = residual_layer(r12)
    r14 = residual_layer(r13)
    r15 = residual_layer(r14)
    r16 = residual_layer(r15)
    r17 = residual_layer(r16)
    r18 = residual_layer(r17)
    r19 = residual_layer(r18)
    r20 = residual_layer(r19)

    r21 = residual_layer(r20)
    r22 = residual_layer(r21)
    r23 = residual_layer(r22)
    r24 = residual_layer(r23)
    r25 = residual_layer(r24)
    r26 = residual_layer(r25)
    r27 = residual_layer(r26)
    r28 = residual_layer(r27)
    r29 = residual_layer(r28)
    r30 = residual_layer(r29)


    r31 = residual_layer(r30)
    r32 = residual_layer(r31)
    r33 = residual_layer(r32)
    r34 = residual_layer(r33)
    r35 = residual_layer(r34)
    r36 = residual_layer(r35)
    r37 = residual_layer(r36)
    r38 = residual_layer(r37)
    r39 = residual_layer(r38)
    r40 = residual_layer(r39)

    output = policy_head(r40)



    model = Model(inputs=[inputs], outputs=[output])

    model.compile(loss=custom_loss_function, optimizer=SGD(lr=0.1, momentum = 0.9))
    
    return model


In [11]:
pi_x = np.load("pi_train_data_ann.npy")
pi_y = np.load("pi_train_label_ann.npy")

dis = np.random.choice(np.shape(pi_x)[0], 20, replace=False)

In [11]:
print dis

[ 688  214 1231  367 1359 1035  703  427 1427  864 1343  481  342  492 1384
 1616 1507  708 1688  265]


In [8]:
pi_x = np.load("pi_train_data_ann.npy")
pi_y = np.load("pi_train_label_ann.npy")


if  os.path.isfile("pi_model_ann.h5"):
        
    pi = load_model('pi_model_ann.h5', custom_objects={'custom_loss_function': custom_loss_function})
else:
    pi = model_pi()


for i in range(3000):
    
    dis = np.random.choice(np.shape(pi_x)[0], 512, replace=False)
    

    pi.fit(pi_x[dis], pi_y[dis], epochs=1, batch_size= 1)
    
pi.save("pi_model_ann.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
512/512 [==============================] - 2962s 6s/step - loss: 5.6970
Epoch 1/1
512/512 [==============================] - 3098s 6s/step - loss: 4.9504
Epoch 1/1
512/512 [==============================] - 2854s 6s/step - loss: 4.2903
Epoch 1/1
512/512 [==============================] - 3054s 6s/step - loss: 3.8005
Epoch 1/1
512/512 [==============================] - 2918s 6s/step - loss: 3.3597
Epoch 1/1
  1/512 [..............................] - ETA: 53:32 - loss: 3.8742

KeyboardInterrupt: 

In [8]:
z_x = np.load("z_train_data_ann.npy")
z_y = np.load("z_train_label_ann.npy")

if  os.path.isfile("z_model_ann.h5"):
        
    z = load_model("z_model_ann.h5")
else:
    z = model_z()


for i in range(3000):
    
    dis = np.random.choice(np.shape(z_x)[0], 512, replace=False)
    

    z.fit(z_x[dis], z_y[dis], epochs=1, batch_size= 1)
    
z.save("z_model_ann.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
512/512 [==============================] - 2970s 6s/step - loss: 50.0887
Epoch 1/1
512/512 [==============================] - 3069s 6s/step - loss: 42.3075
Epoch 1/1
512/512 [==============================] - 2831s 6s/step - loss: 34.7771
Epoch 1/1
512/512 [==============================] - 2832s 6s/step - loss: 28.5512
Epoch 1/1
512/512 [==============================] - 2833s 6s/step - loss: 23.5166
